In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

%env SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
%env TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
!echo $SQLITE_TMPDIR
!echo $TMPDIR
!echo $USERPROFILE

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-icelake/lib/python3.10/site-packages' at start of search paths.
env: SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
env: TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles



In [2]:
import pytorch_lightning
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
import logging
from CPRD.data.database.build_static_db import Static
from CPRD.data.database.build_diagnosis_db import Diagnoses
from CPRD.data.database.build_measurements_and_tests_db import Measurements

torch.manual_seed(1337)
logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}.")

# !export TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles/tmp
!pwd


/rds/bear-apps/2022a/EL8-ice/software/PyTorch/1.12.1-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /dev/shm/branfosj-admin/build-2022a/PyTorch/1.12.1/foss-2022a-CUDA-11.7.0/pytorch-v1.12.1/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Using device: cpu.
/rds/homes/g/gaddcz/Projects/CPRD/data/database


In [3]:
PATH_TO_DB = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/cprd.db"

## Create/load tables

In [6]:
PATH_TO_STATIC = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/baseline/masterDataOptimalWithIMD_v3.csv"
PATH_TO_DIAGNOSIS = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/baseline/masterDataOptimalWithIMD_v3.csv"
PATH_TO_MEASUREMENTS = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/timeseries/measurement_and_tests/lab_measurements/"
load = False

static = Static(PATH_TO_DB, PATH_TO_STATIC, load=load)
diagnosis = Diagnoses(PATH_TO_DB, PATH_TO_DIAGNOSIS, load=load)
measurements = Measurements(PATH_TO_DB, PATH_TO_MEASUREMENTS, load=load)

INFO:root:Creating static_table
INFO:root:Creating diagnosis_table


## Build tables if not loaded

In [ ]:
for table in [static, diagnosis, measurements]:
    print(table)
    if load == False:
        table.build_table(verbose=0, unzip=True, chunksize=1e5)
    print(table)

Static table with 0.00M records.


Building static table: 1it [00:02,  3.00s/it]

Inserted 100000 data owners to the table.


Building static table: 2it [00:05,  2.93s/it]

Inserted 100000 data owners to the table.


...

Building diagnosis table: 282it [19:34,  4.17s/it]
INFO:root:Creating indexes on diagnosis_table


Inserted 55 LYMPHOMA_PREVALENCE_V2 records to the table.
Inserted 4 SYSTEMIC_SCLEROSIS records to the table.
Inserted 56 CHRONICFATIGUESYNDROMEMM_V2 records to the table.
Inserted 4 ADDISONS_DISEASE records to the table.
Inserted 1498 DEATH records to the table.


INFO:root:Creating PRACTICE_PATIENT_ID index on partition_25_Hydroxyvitamin_D2_level_92


Diagnosis table with 51.00M records.


Adding 25_Hydroxyvitamin_D2_level_92                                  : 8it [00:06,  1.32it/s]
INFO:root:Creating PRACTICE_PATIENT_ID index on partition_25_Hydroxyvitamin_D3_level_90
Adding 25_Hydroxyvitamin_D3_level_90                                  : 9it [00:06,  1.48it/s]
INFO:root:Creating PRACTICE_PATIENT_ID index on partition_AST___aspartate_transam_SGOT__46
Adding AST___aspartate_transam_SGOT__46                               : 18it [00:13,  1.35it/s]
INFO:root:Creating PRACTICE_PATIENT_ID index on partition_AST_serum_level_47
Adding AST_serum_level_47                                             : 109it [01:22,  1.32it/s]
INFO:root:Creating PRACTICE_PATIENT_ID index on partition_Albumin___creatinine_ratio_37
Adding Albumin___creatinine_ratio_37                                  : 2it [00:01,  1.47it/s]
INFO:root:Creating PRACTICE_PATIENT_ID index on partition_Basophil_count_22
Adding Basophil_count_22                                              : 869it [11:02,  1.31it/s]
INFO:

In [ ]:
diagnosis.connect()
diagnosis.connection.commit()
print(diagnosis)

## Check query execution time

In [ ]:
import time

measurements.connect()

measurements.cursor.execute("""SELECT
                                   *
                                FROM
                                   sqlite_master
                                ;""")
a = measurements.cursor.fetchall()
for _a in a:
    print(f"{_a[0]}:".ljust(10) + f"{_a[1]}")


In [ ]:

t = time.time()   #  EXPLAIN QUERY PLAN       diagnosis_table            measurement_table        static_table
measurements.cursor.execute(""" EXPLAIN QUERY PLAN 
                                SELECT DISTINCT
                                	PRACTICE_PATIENT_ID
                                FROM
                                	partition_Current_smoker_83
                                 ;""")
a = measurements.cursor.fetchall()
print(a)


# measurements.cursor.execute("""  EXPLAIN QUERY PLAN 
#                                 SELECT
#                                 	*
#                                 FROM
#                                 	measurement_table
#                                 WHERE
#                                     PRACTICE_PATIENT_ID = 'p20515_297525820515' ;""")  # VALUE = '3720368640.0' 
# a = measurements.cursor.fetchall()
# # print(a[:10])
# print(a)
# eval_time = (time.time() - t)
# print(eval_time )

# Collector demo

We can extract unique distinct column values from any table, and use these to chunk the data for processing

* We can add practice level conditions here,
    * e.g. practices that are in the North West region. 

In [ ]:
from CPRD.data.dataset.collector import SQLiteDataCollector
import polars as pl
import logging 
logging.basicConfig(level=logging.INFO)

collector = SQLiteDataCollector(PATH_TO_DB)
collector.connect()

practice_ids = collector._extract_distinct(["static_table"], "PRACTICE_ID", conditions=["HEALTH_AUTH = 'North West'"])

print(f"The first ten unique practice IDs: {practice_ids[:10]} from {len(practice_ids)} total")

## Given these practice IDs, we can chain this query to get the pratice patient ID's in a practice

* We do this as only the static table has all practice_patient_id, patient_id, and practice_id stored and indexed.
    * This means we can very quickly find the keys needed to index on the other tables which only have practice_patient_id indexed.
* We can add patient level conditions here
    * For example, only male, people who died during a period, etc

In [ ]:
practice_patient_ids = []
for p_id in practice_ids:
    practice_patient_ids.append(collector._extract_distinct(["static_table"], "PRACTICE_PATIENT_ID", conditions=[f"PRACTICE_ID = '{p_id}' AND SEX = 'M'"]))
                                
print(f"\nThe first five unique practice_patient IDs from practice {practice_ids[0]}:\n\t{practice_patient_ids[0][:5]} from {len(practice_patient_ids[0])} total")
print(f"\nTotal male patients in each practice:\n\t {[len(ppid) for ppid in practice_patient_ids]}")

### Create a generator which chunks the .db tables by practice_id or practice_patient_id and lazily batch using Polars (Rust)

* We can also choose to pass this method a list of conditions, one for each table.
   * For example, we may only want to collect measurements within some predfined list, period etc

In the first case, we may want to generate by individual level values. 

Here, we then pass in the list we generate over - which in this instance is the list of practice patient ID's belonging to the first chunk. Here we are then yielding one person per call to the generator.

In [ ]:
generator = collector._lazy_generate_by_distinct(practice_patient_ids[0], ["static_table", "measurement_table", "diagnosis_table"], "PRACTICE_PATIENT_ID")
for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(generator):
    break
    
# print(f"Chunk containing practice ID: {chunk_name}\n\n")
display(lazy_table_frames_dict["lazy_static"].collect().head())
display(lazy_table_frames_dict["lazy_measurement"].collect().head())
display(lazy_table_frames_dict["lazy_diagnosis"].collect().head())

Alternatively, we may which to generate groups/batches of individuals for vectorisation. In that case we can pass in a list of lists. We generate over the outer list, and each call yields all the inner list values.

For example, in the below we have an outer list of all practices, and an inner list of patients within the practice. Consequently, each call to generate yields all records of all patients within a practice.

In [ ]:
generator = collector._lazy_generate_by_distinct(practice_patient_ids, ["static_table", "measurement_table", "diagnosis_table"], "PRACTICE_PATIENT_ID")
for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(generator):
    break
    
# print(f"Chunk containing practice ID: {chunk_name}\n\n")
display(lazy_table_frames_dict["lazy_static"].collect().head())
display(lazy_table_frames_dict["lazy_measurement"].collect().head())
display(lazy_table_frames_dict["lazy_diagnosis"].collect().head())


### Test collation of generated lazy frames

In [ ]:
lazy_batch = collector._collate_lazy_tables(**lazy_table_frames_dict)

### View how tables are aligned

In [ ]:
batch = lazy_batch.collect()
display(batch)

first_identifier = list(batch["PRACTICE_PATIENT_ID"].unique())[0]
display(first_identifier)
                        
row = batch.filter(pl.col("PRACTICE_PATIENT_ID") == first_identifier)
display(row)


measur = lazy_table_frames_dict["lazy_measurement"].collect()
row = measur.filter(pl.col("EVENT") == "Diastolic_blood_pressure_5")
display(row)

# Demo wrapper

All of this is wrapped together (without the option to filter - add your own wrapper specific to you application)

In [ ]:
# for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(collector.generator()):
#     lazy_batch = collector._collate_lazy_tables(**lazy_table_frames_dict)
#     break
# print(_idx)
# print(lazy_batch.collect())

In [ ]:
!jupyter nbconvert --to html --no-input buildV2.ipynb